In [6]:
import tensorflow as tf
import cv2

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
image_path = 'C:\PycharmProjects\ETS_Autopilot\\app\static\settings_cache\\img.png'

# Чтение изображения с помощью OpenCV
img = cv2.imread(image_path)

# Получение размеров изображения
height, width, _ = img.shape

# Вывод размеров
print(f"Ширина: {width}, Высота: {height}")

Ширина: 805, Высота: 564


#   Ширина: 800
#   Высота: 600

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.models import Model

def convolutional_block(x, filters, strides=1):
    shortcut = x
    # 1x1 convolution
    x = Conv2D(filters, kernel_size=(1, 1), strides=strides)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # 3x3 convolution
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # 1x1 convolution
    x = Conv2D(4 * filters, kernel_size=(1, 1))(x)
    x = BatchNormalization()(x)
    # Shortcut
    shortcut = Conv2D(4 * filters, kernel_size=(1, 1), strides=strides)(shortcut)
    shortcut = BatchNormalization()(shortcut)
    # Add the shortcut value to the main path
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def identity_block(x, filters):
    shortcut = x
    # 1x1 convolution
    x = Conv2D(filters, kernel_size=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # 3x3 convolution
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # 1x1 convolution
    x = Conv2D(4 * filters, kernel_size=(1, 1))(x)
    x = BatchNormalization()(x)
    # Add the shortcut value to the main path
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def resnet_101(input_shape, num_classes):
    input_data = Input(shape=input_shape)
    # Initial convolutional layer
    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_data)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Residual blocks
    x = convolutional_block(x, filters=64)
    x = identity_block(x, filters=64)
    x = identity_block(x, filters=64)

    x = convolutional_block(x, filters=128, strides=2)
    x = identity_block(x, filters=128)
    x = identity_block(x, filters=128)
    x = identity_block(x, filters=128)

    x = convolutional_block(x, filters=256, strides=2)
    for _ in range(22):
        x = identity_block(x, filters=256)

    x = convolutional_block(x, filters=512, strides=2)
    x = identity_block(x, filters=512)
    x = identity_block(x, filters=512)
    x = GlobalAveragePooling2D()(x)

    # Dense layer for classification
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_data, outputs=x)
    return model

# Create ResNet-101 model
input_shape = (800, 600, 3)
num_classes = 1  # Number of output classes
model = resnet_101(input_shape, num_classes)

# Display model summary
model.summary()
